In [4]:
%pip install lancedb
%pip install pandas
%pip install pyarrow
%pip install ipywidgets
%pip install jupyter


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 925.4 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 1.4 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 2.1 MB/s eta 0:00

In [49]:
import lancedb
import pandas as pd
import pyarrow as pa
import numpy as np

uri = "runtime_assets/vecdb"
db = lancedb.connect(uri)

In [50]:
db.table_names()


['tables']

In [51]:
tbl = db.open_table("tables")
tbl.count_rows()

10

In [52]:
tbl.search(None).select(["content"]).to_pandas().values

array([["- Table: 'Customers' : main table that holds customer data\n  - Columns:\n    - CustomerID (numeric): unique identifier for customer\n    - Name (varchar): customer's full name\n    - Email (varchar): customer contact email\n    - PhoneNumber (varchar): primary contact number\n    - RegionID (numeric): references Regions table\n    - RegistrationDate (date): customer registration date\n    - CustomerType (varchar): customer category (individual, business)"],
       ["- Table: 'Regions' : geographical regions for customer segmentation\n  - Columns:\n    - RegionID (numeric): unique identifier for region\n    - RegionName (varchar): name of the region\n    - Country (varchar): country of the region\n    - TimeZone (varchar): standard time zone"],
       ["- Table: 'Interactions' : log of all customer interactions\n  - Columns:\n    - InteractionID (numeric): unique identifier for interaction\n    - CustomerID (numeric): references Customers table\n    - AgentID (numeric): refere

In [53]:
tbl.search(None).select(["filename"]).to_pandas()

,filename
0,- Table: 'Customers' : main table that holds c...
1,- Table: 'Regions' : geographical regions for ...
2,- Table: 'Interactions' : log of all customer ...
3,- Table: 'Agents' : information about call cen...
4,- Table: 'Departments' : organizational depart...
5,- Table: 'Tickets' : customer support tickets
6,- Table: 'TicketNotes' : additional notes on s...
7,- Table: 'CustomerFeedback' : satisfaction sur...
8,- Table: 'AgentPerformance' : tracking agent p...
9,- Table: 'ServiceLevelAgreements' : SLA tracki...


In [64]:
# generate an embedding request to ollama  on the sentance 
sentence = "where can I find data about customers and their tickets by region"

import requests
import json

def get_embedding(text):
    response = requests.post(
        "http://localhost:11434/api/embeddings",
        json={"model": "nomic-embed-text", "prompt": text}
    )
    return response.json()["embedding"]

question_embedding = get_embedding(sentence)
# normalize the embedding to unit length
question_embedding = question_embedding / np.linalg.norm(question_embedding)


In [65]:
df = tbl.search(question_embedding) \
    .select(["content"]) \
    .limit(3).to_pandas()



In [69]:
rel_tables = "\n".join(df['content'].values)

In [70]:
prompt = f"""
You are an AI assistant that answers questions about database schemas and generates SQL queries based on user questions.
Relevan tables:
{rel_tables}

User Question:
{sentence}

Generate an answer using an explanation or query based on the schema.
"""

print(prompt)


You are an AI assistant that answers questions about database schemas and generates SQL queries based on user questions.
Relevan tables:
- Table: 'Regions' : geographical regions for customer segmentation
  - Columns:
    - RegionID (numeric): unique identifier for region
    - RegionName (varchar): name of the region
    - Country (varchar): country of the region
    - TimeZone (varchar): standard time zone
- Table: 'Customers' : main table that holds customer data
  - Columns:
    - CustomerID (numeric): unique identifier for customer
    - Name (varchar): customer's full name
    - Email (varchar): customer contact email
    - PhoneNumber (varchar): primary contact number
    - RegionID (numeric): references Regions table
    - RegistrationDate (date): customer registration date
    - CustomerType (varchar): customer category (individual, business)
- Table: 'Tickets' : customer support tickets
  - Columns:
    - TicketID (numeric): unique identifier for ticket
    - CustomerID (num

In [71]:
# send request to ollama
response = requests.post(
    "http://localhost:11434/api/generate",
    json={
        "model": "llama3.2",
        "prompt": prompt,
        "options": {
            "temperature": 0.1,
        },
        "stream": False
    }
)

# extract the SQL query from response
sql_query = response.json()["response"]
print("Generated SQL query:")
print(sql_query)


Generated SQL query:
To find data about customers and their tickets by region, you would need to join the 'Customers' table with the 'Tickets' table based on the CustomerID column. Then, you can use a LEFT JOIN to include all regions from the 'Regions' table.

Here's an SQL query that accomplishes this:

```sql
SELECT 
    r.RegionName,
    c.Name,
    t.TicketID,
    t.CreationDate,
    t.Status,
    t.Priority,
    t.Category
FROM 
    Customers c
INNER JOIN 
    Tickets t ON c.CustomerID = t.CustomerID
LEFT JOIN 
    Regions r ON c.RegionID = r.RegionID;
```

This query will return the region name, customer's full name, ticket ID, creation date, status, priority, and category for each customer. The LEFT JOIN ensures that all regions are included in the results, even if there are no customers or tickets associated with a particular region.

If you want to filter the results by a specific region, you can add a WHERE clause:

```sql
SELECT 
    r.RegionName,
    c.Name,
    t.TicketID,